In [23]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse
import matplotlib.pyplot as plt

In [24]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [27]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Podcast_Name', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [28]:
podcast = {}
for i, n in enumerate(train['Podcast_Name'].unique()):
    podcast[n] = i

categorical_map.update(podcast)

In [29]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [30]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [31]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [32]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [33]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [34]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Podcast_Name                                            750000 non-null  float64
 1   Episode_Title                                           750000 non-null  float64
 2   Episode_Length_minutes                                  750000 non-null  float64
 3   Genre                                                   750000 non-null  float64
 4   Host_Popularity_percentage                              750000 non-null  float64
 5   Publication_Day                                         750000 non-null  float64
 6   Publication_Time                                        750000 non-null  float64
 7   Guest_Popularity_percentage                             750000 non-null  float64
 8   Number_of_Ads           

In [35]:
important_features_bygain = ['Episode_Length_minutes','Episode_Title Episode_Length_minutes', 'Episode_Title', 'Podcast_Name','Number_of_Ads',
'Episode_Length_minutes Host_Popularity_percentage', 'Podcast_Name Episode_Length_minutes', 'Host_Popularity_percentage']
idx = []
for i, col in enumerate(x.columns): 
    if col in important_features_bygain: idx.append(i)

In [36]:
x_n = x.iloc[idx]
test_n = test.iloc[idx]

In [37]:
for c in categories: 
    x_n[c] = x_n[c].astype('category')
    test_n[c] = test_n[c].astype('category')

C:\Users\evan\AppData\Local\Temp\ipykernel_27508\4176405769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n[c] = x_n[c].astype('category')
C:\Users\evan\AppData\Local\Temp\ipykernel_27508\4176405769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_n[c] = test_n[c].astype('category')


In [38]:
K = 20
kf = KFold(K, shuffle=True, random_state=55)

In [39]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist", 
                             min_child_weight=10, learning_rate=0.2, n_estimators=1000, importance_type='gain')#, max_depth=5)
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)

[0]	validation_0-rmse:23.20990
[1]	validation_0-rmse:20.23823
[2]	validation_0-rmse:18.06832
[3]	validation_0-rmse:16.51468
[4]	validation_0-rmse:15.43195
[5]	validation_0-rmse:14.68933
[6]	validation_0-rmse:14.18810
[7]	validation_0-rmse:13.85244
[8]	validation_0-rmse:13.62876
[9]	validation_0-rmse:13.47748
[10]	validation_0-rmse:13.37746
[11]	validation_0-rmse:13.31207
[12]	validation_0-rmse:13.26533
[13]	validation_0-rmse:13.23408
[14]	validation_0-rmse:13.21236
[15]	validation_0-rmse:13.19643
[16]	validation_0-rmse:13.18363
[17]	validation_0-rmse:13.17437
[18]	validation_0-rmse:13.16721
[19]	validation_0-rmse:13.16312
[20]	validation_0-rmse:13.15838
[21]	validation_0-rmse:13.15533
[22]	validation_0-rmse:13.15179
[23]	validation_0-rmse:13.14881
[24]	validation_0-rmse:13.14603
[25]	validation_0-rmse:13.14380
[26]	validation_0-rmse:13.14156
[27]	validation_0-rmse:13.14074
[28]	validation_0-rmse:13.13973
[29]	validation_0-rmse:13.13754
[30]	validation_0-rmse:13.13607
[31]	validation_0-

In [41]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [42]:
pred.head()

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
750000,54.404713,53.057514,55.013115,54.413872,54.894985,53.687996,54.189178,54.085377,53.762451,53.972260,52.985615,54.651520,53.411201,54.073944,55.090351,54.289829,53.097076,54.501541,53.311436,52.935070
750001,17.260946,17.882620,17.051388,17.371193,17.832209,17.659971,18.001459,18.085079,17.608267,17.209324,17.125145,17.584599,17.568289,18.108227,17.586315,17.644285,17.612263,18.137104,17.994997,18.231127
750002,50.364288,49.661743,49.491440,49.901356,51.347240,50.190434,51.149273,51.151642,50.297710,51.347778,50.509693,52.312645,51.020573,50.616570,49.886940,49.667801,48.905178,50.062916,50.545441,49.772739
750003,74.660538,75.987045,69.385643,75.427635,73.036598,75.211075,72.830208,70.589180,77.782570,80.233139,74.559456,76.448311,69.244347,76.155106,77.322327,72.212799,72.363266,74.553032,72.691170,72.242470
750004,47.945435,49.735222,49.127796,47.441807,48.315605,48.340557,49.176491,48.256958,48.713139,49.165150,49.004833,48.701099,48.782356,49.277451,47.816975,48.289291,48.753147,48.272728,48.545204,48.902393


In [43]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000    53.991447
750001    17.677740
750002    50.410172
750003    74.146797
750004    48.628181
Name: Listening_Time_minutes, dtype: float32

In [44]:
pred_sum.to_csv('submissions/submission_newparams_20.csv', index_label='id')